First, we define some utility fundtions for saving and loading objects from pickle.
This code can be found in util.py

In [ ]:
import pickle

def save_obj(obj, name ):
    with open( name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)

We now build the text based graph. The code can be found in createSemRepGraph.py

In [ ]:
import pandas as pn
import graph_tool.stats

Defuning some utility functions.

In [ ]:
def getPathMaxPageRank(path):
    pr = []
    for p in path:
        pr.append(pagerank[p])
    return max(pr)


def getPathNames(path):
    pathStrArry = []
    for p in path:
        txt = IdToStrDict[p]
        pathStrArry.append(txt)
    return pathStrArry


def printPageRankHist(semRepGraph):
    prarry = []
    for n in semRepGraph.nodes():
        prarry.append(pagerank[n])


Creating a graph and defining properties.


In [ ]:
semRepGraph = graph_tool.Graph()
text_property = semRepGraph.new_vertex_property("string")
semRepGraph.vertex_properties['text'] = text_property

semType_property = semRepGraph.new_vertex_property("string")
semRepGraph.vertex_properties['semType'] = semType_property

cui_property = semRepGraph.new_vertex_property("string")
semRepGraph.vertex_properties['cui'] = cui_property

We now read a csv file containing the PREDICATE table from SemRep (see: https://semrep.nlm.nih.gov/).
We shall add vertoces and edges to the graph based on this table.

In [ ]:
predicates = pn.read_csv('/my_folder/SemRep.txt') 
predicates.columns = ['PREDICATION_ID', 'SENTENCE_ID', 'PMID', 'PREDICATE', 'SUBJECT_CUI', 'SUBJECT_NAME',
                      'SUBJECT_SEMTYPE',
                      'SUBJECT_NOVELTY', 'OBJECT_CUI', 'OBJECT_NAME', 'OBJECY_SEMTYPE', 'OBJECT_NOVELTY']

In [ ]:
predTypes = ['CAUSES']
predicates = predicates[predicates['PREDICATE'].isin(predTypes)]

Data can also be filtered by OBJECT_SEMTYPE and SUBJECT_SEMTYPE.
For example:

In [ ]:
semtypes=['orgm','bpoc','diap','ortf','bsoj','inpo','tisu','topp','mamm','inpr','geoa','hlca','bdsy','blor','hcro','lbpr'
          ,'inbe','orga','menp','mnob','humn','amph','plnt','spco','anim','resa','anab','eehu','tmco','edac','ftcn','ocdi',
          'dora','qnco','orgt','npop','qlco','podg','prog','bird','mcha','rept','fish','phob','socb','idcn','popg',
          'bmod','emod','aggp','famg','rnlw','mbrt','pros','lang','ocac','gora','medd']

predicates = predicates[predicates['OBJECT_SEMTYPE'].isin(semtypes)==False]
predicates = predicates[predicates['SUBJECT_SEMTYPE'].isin(semtypes)==False]


In [ ]:
nodeDict = {}

subjects = predicates['SUBJECT_NAME'].unique()
objects = predicates['OBJECT_NAME'].unique()
nodes = list(set(subjects).union(set(objects)))
for n in nodes:
    newNode = semRepGraph.add_vertex()
    text_property[newNode] = n
    nodeDict[n] = int(newNode)

We create an edge for each 

In [ ]:
index = 0
edges = []
while index < len(predicates):
    print(index)
    object = nodeDict[predicates.iloc[index]['SUBJECT_NAME']]
    subject = nodeDict[predicates.iloc[index]['OBJECT_NAME']]
    edges.append((object, subject))
    index += 1

semRepGraph.add_edge_list(edges)

graph_tool.stats.remove_self_loops(semRepGraph)


In [ ]:
semRepGraph.save("/my_folder/semRepGraph.xml")  # save garph
save_obj(nodeDict, "/my_folder/nodeDict")  # save dictionary

Read graph and dictionary 

In [ ]:
semRepGraph = graph_tool.load_graph("/my_folder/semRepGraph.xml")
nodeDict = load_obj("/my_folder/nodeDict")

In [ ]:
# init IdTpStrDict
IdToStrDict = {}
for k in nodeDict.keys():
    IdToStrDict[nodeDict[k]] = k

In [ ]:
s = nodeDict['Hydroxymethylglutaryl-CoA Reductase Inhibitors']
t = nodeDict['Blood Pressure']

Find paths. In this example, we look at paths with cutoff=2

In [ ]:
paths = graph_tool.topology.all_paths(semRepGraph, s, t, cutoff=2)

We can now go over the paths.
While we go over them we can "translate" the node names to text.
We can also rank or filter the paths  the paths. In this example, we calculate the max pagerank for each path.

In [ ]:
pagerank = graph_tool.centrality.pagerank(semRepGraph) #for ranking


textpaths = pn.DataFrame()
prarray = []  # will hold max page rank for each path
for p in paths:
    prarray.append(getPathMaxPageRank(p))
    textpaths = textpaths.append(pn.DataFrame([getPathNames(p)]))

textpaths = textpaths.drop_duplicates()

textpaths now holds all the reasoning paths.

In [ ]:
textpaths.head()
